In [ ]:
!pip install -q diffusers transformers xformers accelerate
!pip install -q numpy scipy ftfy Pillow
#!pip install super-image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 13.9 MB/s eta 0:00:00
   ━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.2/899.7 MB 28.6 MB/s eta 0:00:27

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import torch
import numpy as np
import os

import time

from PIL import Image
from IPython import display as IPdisplay
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from diffusers import StableDiffusionPipeline
from diffusers import DiffusionPipeline
from diffusers import (
    DDIMScheduler,
    PNDMScheduler,
    LMSDiscreteScheduler,
    DPMSolverMultistepScheduler,
    EulerAncestralDiscreteScheduler,
    EulerDiscreteScheduler,
)
from transformers import logging
import glob

logging.set_verbosity_error()

In [ ]:
print(torch.cuda.is_available())

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

In [ ]:
model_name_or_path = "runwayml/stable-diffusion-v1-5"

scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000, steps_offset=1)


pipe = StableDiffusionPipeline.from_pretrained(
    model_name_or_path,
    scheduler=scheduler,
    torch_dtype=torch.float32,
    safety_checker = None,
).to(device)

In [ ]:
# efficiency settings
pipe.enable_model_cpu_offload()
pipe.unet.to(memory_format=torch.channels_last)
pipe.vae.enable_slicing()
pipe.vae.enable_tiling()
pipe.enable_xformers_memory_efficient_attention()

In [ ]:
# The seed is set to "None", because we want different results each time we run the generation.
seed = 0

if seed is not None:
    generator = torch.manual_seed(seed)
else:
    generator = None

# The guidance scale is set to its normal range (7 - 10).
guidance_scale = 7

# How long should the thing bake for
num_inference_steps = 120


# I would not recommend less than 512 on either dimension. This is because this model was trained on 512x512 image resolution.
height = 960
width = 512

latents = torch.randn(
    (1, pipe.unet.config.in_channels, height // 8, width // 8),
    generator=generator,
)

In [ ]:
def generate_single_image(positive_prompt_embeds, negative_prompt_embeds, height, width, num_inference_steps, generator, latents):
  image = pipe(
                height=height,
                width=width,
                num_images_per_prompt=1,
                prompt_embeds=positive_prompt_embeds,
                negative_prompt_embeds=negative_prompt_embeds,
                num_inference_steps=num_inference_steps,
                guidance_scale=guidance_scale,
                generator=generator,
                latents=latents,
                     ).images
  return image

def plot_images(images):
    plt.figure(figsize=(10, 10))
    for i in range(len(images)):
        ax = plt.subplot(1, len(images), i + 1)
        plt.imshow(images[i])
        plt.axis("off")



In [ ]:
# The text prompt that describes the desired output image.
prompt_one =  "a massive lush alien planet seen from deep space, emerald oceans and turquoise seas, vast swirling cloud systems, glowing auroras wrapping the poles, dense green continents with golden coastlines, soft sunlight illuminating the atmosphere, cinematic wide-angle view, ultra-detailed planetary textures, realistic light scattering, astrophotography style, NASA-quality realism, 8k resolution, epic scale, serene and majestic"
prompt_two =  "a colossal volcanic planet floating in outer space, cracked obsidian surface with flowing lava rivers, fiery magma glowing through tectonic fractures, thick ash clouds and sulfur storms, intense red and orange color palette, dramatic rim lighting, high contrast shadows, ultra-realistic planetary geology, extreme detail, cinematic sci-fi realism, deep space background with distant stars"
prompt_three = "a frozen exoplanet viewed from orbit, vast ice-covered surface with jagged glaciers and crystalline ice plates, pale blue and white tones, thin misty atmosphere, soft reflected starlight, subtle cloud bands, extreme planetary scale, hyper-realistic textures, astrophysical accuracy, calm and mysterious mood, ultra-sharp focus, space photography"
prompt_four = "a massive exotic gas planet in outer space, swirling atmospheric bands of violet, cyan, and amber, luminous storms and massive cyclones, glowing upper atmosphere, smooth volumetric clouds, dramatic lighting, ultra-detailed atmospheric physics, deep space backdrop, cinematic composition, breathtaking scale, realistic sci-fi art, 8k clarity"
negative_prompt = "low resolution, blurry, flat lighting, overexposed, underexposed, washed out colors, dull composition, visible pixels, jpeg artifacts, noisy image, cartoon style, anime, fantasy illustration, hand-drawn, text, watermark, logo, frame, border, distorted planet shape, unrealistic proportions, duplicated planets, low detail surface, messy clouds"

def generate_prompt_vector(prompt):
  prompt_tokens = pipe.tokenizer(prompt,
    padding="max_length",
    max_length=pipe.tokenizer.model_max_length,
    truncation=True,
    return_tensors="pt",)

  prompt_embeds = pipe.text_encoder(prompt_tokens.input_ids.to(device))[0]

  return prompt_embeds

prompt_one_embeds = generate_prompt_vector(prompt_one)
prompt_two_embeds = generate_prompt_vector(prompt_two)
prompt_three_embeds = generate_prompt_vector(prompt_three)
prompt_four_embeds = generate_prompt_vector(prompt_four)
negative_prompt_embeds = generate_prompt_vector(negative_prompt)

In [ ]:
image = generate_single_image(prompt_one_embeds, negative_prompt_embeds, height, width, num_inference_steps, generator, latents)
plot_images(image)


In [ ]:
image = generate_single_image(prompt_two_embeds, negative_prompt_embeds, height, width, num_inference_steps, generator, latents)
plot_images(image)

In [ ]:
image = generate_single_image(prompt_three_embeds, negative_prompt_embeds, height, width, num_inference_steps, generator, latents)
plot_images(image)

In [ ]:
image = generate_single_image(prompt_four_embeds, negative_prompt_embeds, height, width, num_inference_steps, generator, latents)
plot_images(image)

In [ ]:
def slerp(t, v0, v1, DOT_THRESHOLD=0.9995):
    v0 = v0.detach().cpu().numpy()
    v1 = v1.detach().cpu().numpy()

    #calculate dot product of vectors
    dot = np.sum(v0 * v1 / (np.linalg.norm(v0) * np.linalg.norm(v1)))
    #if transformation would not be meaningful, do linear interpolation
    if np.abs(dot) > DOT_THRESHOLD:
        v2 = (1 - t) * v0 + t * v1
    # otherwise, spheical interpolate
    else:
        theta_0 = np.arccos(dot)
        sin_theta_0 = np.sin(theta_0)
        theta_t = theta_0 * t
        sin_theta_t = np.sin(theta_t)
        s0 = np.sin(theta_0 - theta_t) / sin_theta_0
        s1 = sin_theta_t / sin_theta_0
        v2 = s0 * v0 + s1 * v1

        v2 = torch.tensor(v2)

    return v2


In [ ]:
num_interpolation_steps = 60

interpolation_list = []
for _, t in enumerate(np.linspace(0, 1, num_interpolation_steps)):
    interpolation_list.append(slerp(float(t), prompt_one_embeds, prompt_two_embeds))
for _, t in enumerate(np.linspace(0, 1, num_interpolation_steps)):
    interpolation_list.append(slerp(float(t), prompt_two_embeds, prompt_three_embeds))
for _, t in enumerate(np.linspace(0, 1, num_interpolation_steps)):
    interpolation_list.append(slerp(float(t), prompt_three_embeds, prompt_four_embeds))
for _, t in enumerate(np.linspace(0, 1, num_interpolation_steps)):
    interpolation_list.append(slerp(float(t), prompt_four_embeds, prompt_one_embeds))

In [ ]:
images = []
# Disable image generation progress bar, we'll display our own
pipe.set_progress_bar_config(disable=True)
print(range(len(interpolation_list)))
for batch in tqdm(range(len(interpolation_list))):
  images.append(pipe(
                height=height,
                width=width,
                num_images_per_prompt=1,
                prompt_embeds=interpolation_list[batch],
                negative_prompt_embeds=negative_prompt_embeds,
                num_inference_steps=num_inference_steps,
                guidance_scale=guidance_scale,
                generator=generator,
                latents=latents,
                     ).images

  )

In [ ]:
def glob_save_images(base_directory, image_library, image_counter):
  file_count = len(image_library)
  globbed_dir_num = int((file_count/100)+1)
  glob_file = 0
  for flc in range(0, globbed_dir_num):
      os.makedirs(base_directory + "%05d" % (flc+1), exist_ok=True)
  for idx in tqdm(range(len(image_library))):
    if idx  >= 0 and idx <= 100:
         plt.figure(figsize=(10,10))
         plt.imshow(image_library[idx][0])
         plt.axis('off')
         filename = base_directory + "%05d/generated_plot_%05s.png" % ((glob_file+1), (image_counter+1))
         plt.savefig(filename, transparent=True,bbox_inches="tight",pad_inches=0.0 )
         plt.close()
         image_counter += 1
    if idx >= 101 and idx <= 200:
         plt.figure(figsize=(10,10))
         plt.imshow(image_library[idx][0])
         plt.axis('off')
         filename = base_directory + "%05d/generated_plot_%05d.png" % ((glob_file+2), (image_counter+1))
         plt.savefig(filename, transparent=True,bbox_inches="tight",pad_inches=0.0 )
         plt.close()
         image_counter += 1
    if idx >= 201 and idx <= 300:
         plt.figure(figsize=(10,10))
         plt.imshow(image_library[idx][0])
         plt.axis('off')
         filename = base_directory + "%05d/generated_plot_%05d.png" % ((glob_file+3),( image_counter+1))
         plt.savefig(filename, transparent=True,bbox_inches="tight",pad_inches=0.0 )
         plt.close()
         image_counter += 1
    if idx >= 301 and idx <= 400:
         plt.figure(figsize=(10,10))
         plt.imshow(image_library[idx][0])
         plt.axis('off')
         filename = base_directory + "%05d/generated_plot_%05d.png" % ((glob_file+4), (image_counter+1))
         plt.savefig(filename, transparent=True,bbox_inches="tight",pad_inches=0.0 )
         plt.close()
         image_counter += 1
    if idx >= 401 and idx <= 500:
         plt.figure(figsize=(10,10))
         plt.imshow(image_library[idx][0])
         plt.axis('off')
         filename = base_directory + "%05d/generated_plot_%05d.png" % ((glob_file+5), (image_counter+1))
         plt.savefig(filename, transparent=True,bbox_inches="tight",pad_inches=0.0 )
         plt.close()
         image_counter += 1
    if idx >= 501 and idx <= 600:
         plt.figure(figsize=(10,10))
         plt.imshow(image_library[idx][0])
         plt.axis('off')
         filename = base_directory + "%05d/generated_plot_%05d.png" % ((glob_file+6), (image_counter+1))
         plt.savefig(filename, transparent=True,bbox_inches="tight",pad_inches=0.0 )
         plt.close()
         image_counter += 1
    if idx >= 601 and idx <= 700:
         plt.figure(figsize=(10,10))
         plt.imshow(image_library[idx][0])
         plt.axis('off')
         filename = base_directory + "%05d/generated_plot_%05d.png" % ((glob_file+7), (image_counter+1))
         plt.savefig(filename, transparent=True,bbox_inches="tight",pad_inches=0.0 )
         plt.close()
         image_counter += 1
    if idx >= 701 and idx <= 800:
         plt.figure(figsize=(10,10))
         plt.imshow(image_library[idx][0])
         plt.axis('off')
         filename = base_directory + "%05d/generated_plot_%05d.png" % ((glob_file+8), (image_counter+1))
         plt.savefig(filename, transparent=True,bbox_inches="tight",pad_inches=0.0 )
         plt.close()
         image_counter += 1
    if idx >= 801 and idx <= 900:
         plt.figure(figsize=(10,10))
         plt.imshow(image_library[idx][0])
         plt.axis('off')
         filename = base_directory + "%05d/generated_plot_%05d.png" % ((glob_file+9), (image_counter+1))
         plt.savefig(filename, transparent=True,bbox_inches="tight",pad_inches=0.0 )
         plt.close()
         image_counter += 1
    if idx >= 901 and idx <= 1000:
         plt.figure(figsize=(10,10))
         plt.imshow(image_library[idx][0])
         plt.axis('off')
         filename = base_directory + "%05d/generated_plot_%05d.png" % ((glob_file+10), (image_counter+1))
         plt.savefig(filename, transparent=True,bbox_inches="tight",pad_inches=0.0 )
         plt.close()
         image_counter += 1
    if idx >= 1001 and idx <= 1100:
         plt.figure(figsize=(10,10))
         plt.imshow(image_library[idx][0])
         plt.axis('off')
         filename = base_directory + "%05d/generated_plot_%05d.png" % ((glob_file+11), (image_counter+1))
         plt.savefig(filename, transparent=True,bbox_inches="tight",pad_inches=0.0 )
         plt.close()
         image_counter += 1


In [ ]:
counter = 0
base_directory = '/content/gdrive/My Drive/stable_diffusion/'
glob_save_images(base_directory, images, counter)

In [ ]:
import os
import cv2

In [ ]:
image_base_folder = '/content/gdrive/My Drive/stable_diffusion/' # make sure to use your folder
file_count = len(os.listdir(image_base_folder))
print(file_count)

video_name = 'test_thousands.avi'
os.chdir("/content/gdrive/My Drive/stable_diffusion_outputs/")

image_locations = []
for dir in range(1, file_count+1):
  for image in os.listdir(image_base_folder + "%05d/" % (dir)):
    if image.endswith(".jpg") or image.endswith(".jpeg") or image.endswith("png"):
      image_locations.append(image_base_folder + "%05d/" % (dir) + image)

images = [img for img in image_locations
              if img.endswith(".jpg") or
                 img.endswith(".jpeg") or
                 img.endswith("png")]
frame = cv2.imread(images[0])

height, width, layers = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'XVID')
video = cv2.VideoWriter(video_name, fourcc, 4,(width, height))

for image in images:
    video.write(cv2.imread(image))

video.release()

cv2.destroyAllWindows()

In [ ]:
#import cv2
#import numpy as np
#video_path = "/content/gdrive/My Drive/stable_diffusion_outputs/test_thousands.avi"
#cap = cv2.VideoCapture(video_path)
#ret = True
#frames = []
#while ret:
#    ret, img = cap.read()
#    if ret:
#      recolored_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#      frames.append(recolored_img)
#
#video = np.stack(frames, axis=0)
#
#import gc
#from super_image import EdsrModel, ImageLoader
#model = EdsrModel.from_pretrained('eugenesiow/edsr-base', scale=2)
#
#images_upscaled = []
#with torch.no_grad():
#  for frame in tqdm(range(len(video))):
#    frame_float = video[frame].astype(np.float32)
#    image_tensor = torch.from_numpy(frame_float)
#    image_tensor = image_tensor.permute(2, 0, 1)
#    upscale_image = model(image_tensor)
#    upscale_image = upscale_image.permute(1, 2, 0)
#    upscale_image = upscale_image.cpu().numpy()
#    upscale_image = cv2.cvtColor(upscale_image, cv2.COLOR_RGB2BGR)
#    images_upscaled.append(upscale_image)
#    del upscale_image
#    del image_tensor
#    del frame_float
#    gc.collect()
#    torch.cuda.empty_cache()
#
#counter = 0
#base_directory = '/content/gdrive/My Drive/upscaled_images/'
#glob_save_images(base_directory, images_upscaled, counter)

In [ ]:
#del images_upscaled
#gc.collect()
#torch.cuda.empty_cache()

In [ ]:
#image_base_folder = '/content/gdrive/My Drive/upscaled_images/' # make sure to use your folder
#file_count = len(os.listdir(image_base_folder))
#print(file_count)

#video_name = 'test_thousands_upscaled.avi'
#os.chdir("/content/gdrive/My Drive/stable_diffusion_outputs/")

#image_locations = []
#for dir in range(1, file_count+1):
#  for image in os.listdir(image_base_folder + "%05d/" % (dir)):
#    if image.endswith(".jpg") or image.endswith(".jpeg") or image.endswith("png"):
#      image_locations.append(image_base_folder + "%05d/" % (dir) + image)

#images = [img for img in image_locations
#              if img.endswith(".jpg") or
#                 img.endswith(".jpeg") or
#                 img.endswith("png")]

#frame = cv2.imread(images[0])

#height, width, layers = frame.shape

#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#video = cv2.VideoWriter(video_name, fourcc, 34,(width, height))

#for image in images:
#    video.write(cv2.imread(image))

#video.release()  # releasing the video generated

#cv2.destroyAllWindows()

In [ ]:
fileList = glob.glob('/content/gdrive/My Drive/stable_diffusion/*/*')
print("Number of files removed ",len(fileList))

for filePath in fileList:
  os.remove(filePath)

In [ ]:
from google.colab import runtime
runtime.unassign()
